In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency

In [2]:
#chargements des caractéristiques
carac = pd.read_csv('carcteristiques-2022.csv', sep=';')
#conversion en chaine
cols_to_cast = ['Accident_Id', 'lum', 'agg', 'int', 'atm', 'col']
carac[cols_to_cast] = carac[cols_to_cast].astype('int')
#conversion des latitudes/longitudes
carac['lat']= carac['lat'].str.replace(',','.').astype('float')
carac['long']= carac['long'].str.replace(',','.').astype('float')

# suppression de l'adresse qui ne sert à rien
carac = carac.drop('adr', axis = 1)

#on ajoute un créneau horaire
carac['hh'] = carac['hrmn'].str[:2]

# traitement des -1 en NA
carac.replace({-1:np.nan}, inplace=True)

# gestion des NAs
# lum : 2 nas => valuers la plus fréquentes (1)
carac.loc[carac['lum'].isna(),'lum'] = 1

# int : 7 nas => valuers la plus fréquentes (1)
carac.loc[carac['int'].isna(),'int'] = 1

# atm : 1 nas => valuers la plus fréquentes (1)
carac.loc[carac['atm'].isna(),'atm'] = 1

# col : 63 nas => valuers la plus fréquentes (6) : atre collision
carac.loc[carac['col'].isna(),'col'] = 6

In [123]:
# chargement des usagers
usagers = pd.read_csv('usagers-2022.csv', sep=';')

#suppression du num_veh qui est en doublon avec id_vehicule
usagers.drop('num_veh', axis = 1, inplace = True)

#correction dees valeurs inconnue actp
usagers.loc[usagers['actp'].isin([' -1','8','9']),'actp'] = -1

# place manquante : tous les usagers concernés sont conduxcteurs : donc on remplace par 1
usagers.loc[usagers['place']== -1 ,'place'] = 1

# catu : on remplca e0 par 1 = le plus nombreux
#usagers.loc[usagers['catu']== 0 ,'catu'] = 1
usagers['catu'] = usagers['catu'].replace(0,1)


# gravité : bcp d'information manquantes pour ces usagers : on supprime
usagers.dropna(subset=['grav'], inplace=True)

#sexe : le plus courant = H
usagers['sexe'] = usagers['sexe'].replace({0:-1})
usagers.loc[usagers['sexe']== -1 ,'sexe'] = 1

# trajet :  catégorie 0 : non renseignée conservée car très nombreuse
usagers.loc[usagers['trajet']== -1, 'trajet'] = 0



#secu  : un champ binaire par équipement
for i in range(1,10) :
    usagers['secu_'+str(i)] = ((usagers['secu1']==i) | (usagers['secu2']==i) | (usagers['secu3']==i))*1

usagers = usagers.drop(['secu1','secu2','secu3'], axis = 1)



#année de naissance : année moyenne
usagers.loc[usagers['an_nais'] == 0,'an_nais'] = 1987
usagers.loc[usagers['an_nais'].isna(),'an_nais'] = 1987

#sexe : 1 est le plus nombreux
usagers.loc[usagers['sexe'] == 0,'sexe'] = 1

# locp. pour les usagers non piétons, la valeur -1 est remplacé"e par 0 sans objet
# pour les piétonzs, on met 9 : inconnue
usagers.loc[usagers['catu']!=3,'locp'] = 0
usagers.loc[(usagers['catu']==3) & (usagers['locp']==-1),'locp'] = 9

#actp. pour les non piétons, valeurmanquantes = 0 pour sans objet
# pour les piétons : B = inconnu
usagers.loc[usagers['catu']!=3,'actp'] = 0
usagers.loc[(usagers['catu']==3) & (usagers['actp'] == -1),'actp'] = 'B'

#on recode actp A et B en 10 ,11
usagers.loc[usagers['actp']=='A', 'actp'] = '10'
usagers.loc[usagers['actp']=='B', 'actp'] = '11'

#etatp : pour les non piétons, on mets à 0 = sans objet
#pour le smiétons, 1 (le plus fréquent)
usagers.loc[(usagers['catu']!=3)&(usagers['etatp']==-1), 'etatp'] = 0
usagers.loc[(usagers['catu']==3)&(usagers['etatp']==-1), 'etatp'] = 1

#cast
usagers[['an_nais','actp']] = usagers[['an_nais','actp']].astype('int')
